In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
#from tqdm.auto import tqdm
from tqdm import tqdm
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in tqdm(os.listdir(maindir + '\\testing_modules')):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)
plt.close('All')
%matplotlib qt

100%|██████████| 22/22 [00:14<00:00,  1.47it/s]


In [2]:
from scipy.optimize import curve_fit
from scipy.stats import mode

In [65]:
det_sets = '_hr_4x4_'

scanid = 152663
filedir = '''D:\\Musterman_postdoc\\20240125_Musterman\\'''
map_filename = f'scan{scanid}_dexela_xrd.tif'
h5_filename = f'scan{scanid}_xrd.h5'

test1 = XRDMap.from_image_stack(map_filename, wd=filedir + 'amended\\', energy=21, scanid=scanid, save_hdf=False)
test1.set_calibration('scan152664_calibration.poni', filedir=filedir)

Loading images...done!


In [66]:
dexela2315 = Dexela2315()
dexela2315.set_binning(bin_size=(4, 4))
test1.ai.detector = dexela2315
test1.ai._cached_array = {}

In [67]:
dwell = 1

dark_dir = '''D:\\Musterman_postdoc\\20240125_Musterman\\dark_fields\\fit_images\\'''
dark_paths = np.array(os.listdir(dark_dir))
offset_mask = [det_sets in x and 'offset' in x for x in dark_paths]
offset_df = io.imread(f'{dark_dir}{dark_paths[offset_mask][0]}')
scale_mask = [det_sets in x and 'slope' in x for x in dark_paths]
slope_df = io.imread(f'{dark_dir}{dark_paths[scale_mask][0]}')

dark_field = slope_df * dwell + offset_df
#scaled_df = dark_field * np.median(test.map.min_image) / np.median(dark_field)
test1.map.correct_dark_field(dark_field=dark_field.astype(np.float32))

Correcting dark-field...done!


In [68]:
scanid = 152673
filedir = '''D:\\Musterman_postdoc\\20240125_Musterman\\'''
map_filename = f'scan{scanid}_dexela_xrd.tif'
h5_filename = f'scan{scanid}_xrd.h5'

test2 = XRDMap.from_image_stack(map_filename, wd=filedir + 'amended\\', energy=21, scanid=scanid, save_hdf=False)
test2.set_calibration('scan152667_calibration.poni', filedir=filedir)

Loading images...done!


In [69]:
dexela2315 = Dexela2315()
dexela2315.set_binning(bin_size=(4, 4))
test2.ai.detector = dexela2315
test2.ai._cached_array = {}

In [70]:
dwell = 1

dark_dir = '''D:\\Musterman_postdoc\\20240125_Musterman\\dark_fields\\fit_images\\'''
dark_paths = np.array(os.listdir(dark_dir))
offset_mask = [det_sets in x and 'offset' in x for x in dark_paths]
offset_df = io.imread(f'{dark_dir}{dark_paths[offset_mask][0]}')
scale_mask = [det_sets in x and 'slope' in x for x in dark_paths]
slope_df = io.imread(f'{dark_dir}{dark_paths[scale_mask][0]}')

dark_field = slope_df * dwell + offset_df
#scaled_df = dark_field * np.median(test.map.min_image) / np.median(dark_field)
test2.map.correct_dark_field(dark_field=dark_field.astype(np.float32))

Correcting dark-field...done!


In [71]:
test1.map.apply_solidangle_correction(ai=test1.ai)
test2.map.apply_solidangle_correction(ai=test2.ai)

Applying solid angle correction...

done!
Applying solid angle correction...done!


In [74]:
mask1 = test1.map.min_image >= 15
mask2 = test2.map.min_image >= 6

mask1 = gaussian_filter(test1.map.min_image, sigma=1) >= 0
mask2 = gaussian_filter(test2.map.min_image, sigma=1) >= 0

mask1 = expand_labels(mask1, 2)
mask2 = expand_labels(mask2, 2)

#mask1 = test1.map.min_image >= 1
#mask2 = test2.map.min_image >= 1

In [76]:
test1.plot_image(mask2)

In [77]:
flat1 = test1.map.med_image.copy()
flat2 = test2.map.med_image.copy()
mean1 = np.mean(flat1)
std1 = np.std(flat1)
mean2 = np.mean(flat2)
std2 = np.std(flat2)

norm1 = (flat1 - np.mean(flat1)) / std1
norm2 = (flat2 - np.mean(flat2)) / std2

norm1[mask1] = norm2[mask1]
norm2[mask2] = norm1[mask2]

new_flat1 = norm1 * std1 + mean1
new_flat2 = norm2 * std2 + mean2

#flat1[mask1] = flat2[mask1]
#flat2[mask2] = flat1[mask2]

'''flat1 = flat1 / (np.mean(flat1) - np.min(flat1))
flat1 = flat1 - np.mean(flat1) + 1

flat2 = flat2 / (np.mean(flat2) - np.min(flat2))
flat2 = flat2 - np.mean(flat2) + 1'''

#flat1 -= np.min(flat1)
#flat2 -= np.min(flat2)

'flat1 = flat1 / (np.mean(flat1) - np.min(flat1))\nflat1 = flat1 - np.mean(flat1) + 1\n\nflat2 = flat2 / (np.mean(flat2) - np.min(flat2))\nflat2 = flat2 - np.mean(flat2) + 1'

In [78]:
test1.plot_image(new_flat1)

In [79]:
test2.plot_image(new_flat2)

In [80]:
new_flat1[new_flat1 <= 0] = gaussian_filter(new_flat1, sigma=1)[new_flat1 <= 0]
new_flat2[new_flat2 <= 0] = gaussian_filter(new_flat2, sigma=1)[new_flat2 <= 0]

In [152]:
corr_test1 = test1.map.images.copy() / flat1

In [154]:
test1.plot_image([0, 0])

In [81]:
filename = f'{filedir}flat_fields\\flat_field{det_sets}.tif'
io.imsave(filename, new_flat1, check_contrast=False)

In [17]:
filename = f'{filedir}dark_fields\\median_images\\scan{test.scanid}_ln_4x4_2.0s.tif'
io.imsave(filename, test.map.med_image, check_contrast=False)

In [28]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(test.map.sum_map)
fig.colorbar(im, ax=ax)

plt.show()

In [13]:
test.plot_image(test.map.med_image)

In [19]:
test.map.dtype = np.float32

In [27]:
test.plot_interactive_map(img_vmin=0, img_vmax=400)

In [40]:
test.plot_image(test.map.med_image)

In [41]:
test.plot_image(test.map.mean_image)

In [44]:
test.plot_image(test.map.mean_image - test.map.med_image)

In [9]:
any(file_mask)

False

In [35]:
filedir = '''D:\\Musterman_postdoc\\20240125_Musterman\\'''
filebase = f'{filedir}dark_fields\\median_images\\'
files = os.listdir(filebase)

det_sets = '_hr_1x1_'
dwell_strings = [
    '2.0',
    '1.5',
    '1.0',
    '0.75',
    '0.5',
    '0.25',
    '0.1'
]

dwell_times = np.array([float(x) for x in dwell_strings])

image_list = []
for dwell in dwell_strings:
    file_mask = [f'{det_sets}{dwell}' in x for x in files]
    image_name = files[np.where(file_mask)[0][0]]
    image = io.imread(filebase + image_name)
    image_list.append(image)

image_list = np.asarray(image_list, dtype=np.float32)

In [30]:
def line(x, slope, offset):
    return slope * x + offset

In [36]:
# This is faster than the delayed parallelized code...
p0 = [50, 320] # slope and offset
slope_list = []
offset_list = []
r_squared_list = []

for pixel in tqdm(image_list.reshape(image_list.shape[0], -1).T):
    popt, _ = curve_fit(line, dwell_times, pixel, p0=p0)
    r_squared = compute_r_squared(pixel, line(dwell_times, *popt))
    
    slope_list.append(popt[0])
    offset_list.append(popt[1])
    r_squared_list.append(r_squared)

100%|██████████| 5971968/5971968 [13:30<00:00, 7369.95it/s]


In [24]:
Dexela2315().max_shape

(1944, 3072)

In [37]:
slope_image = np.asarray(slope_list).reshape(np.array(Dexela2315().max_shape) // 1)
offset_image = np.asarray(offset_list).reshape(np.array(Dexela2315().max_shape) // 1)
r_squared_image = np.asarray(r_squared_list).reshape(np.array(Dexela2315().max_shape) // 1)

In [38]:
test.plot_image(slope_image)

In [39]:
io.imsave(f'{filedir}dark_fields\\fit_images\\scan{test.scanid}{det_sets}slope.tif', slope_image, check_contrast=False)
io.imsave(f'{filedir}dark_fields\\fit_images\\scan{test.scanid}{det_sets}offset.tif', offset_image, check_contrast=False)
io.imsave(f'{filedir}dark_fields\\fit_images\\scan{test.scanid}{det_sets}r_squared.tif', r_squared_image, check_contrast=False)